# W2 Exercise - Align CARD Resistance Genes to Escherichia Coli Genomes


#### Throughout the exercise, there are blanks you will need to fill in, these blanks are highlighted with three dashes `---`

### Imports

In [10]:
import os
import subprocess

import pandas as pd
import pysam
from Bio import SeqIO

### 1. Use Subprocess to build a Bowtie Index for some Sample Genomes

- Bowtie2 is an alignment tool installed through Biopython
- It allows you to efficiently search for small genomic sequences within larger sequences
- To do this Bowtie creates an INDEX of the sequences to search within (in our case the genome assemblies)
- You will use bowtie to search for CARD Resistance sequences in a few example Genomes

Documentation for Bowtie2 building an Index: https://bowtie-bio.sourceforge.net/bowtie2/manual.shtml#the-bowtie2-build-indexer

In [3]:
# Set up filepaths to access genomes and make empty directories for outputs
all_genome_fastas = [x for x in os.listdir('../data/genomes_subset/') if x.endswith('.fna')]
os.makedirs('../data/genome_index/', exist_ok=True)
os.makedirs('../data/alignments/', exist_ok=True)

# For 5 genomes - build a bowtie index for each one to allow efficient searching
for fasta in all_genome_fastas[0:5]:
    
    # Get just the sequencing ID from the FASTA filepath
    seq_id = fasta.replace('.fna','')

    # Set up filepaths (absolute to account for different locations)
    genome_path = '../data/genomes_subset/' + fasta
    genome_abspath = os.path.abspath(genome_path)
    output_path = '../data/genome_index/' + seq_id
    output_abspath = os.path.abspath(output_path)

    # Use subprocess to build an index per sample genome
    # Hint: the subprocess will be [bowtie command (see documentation), path_to_genomes, path_to_output]
    subprocess.run(
        ['bowtie2-build', genome_abspath, output_path],
        capture_output=False, 
        stdout=subprocess.DEVNULL, 
        stderr=subprocess.STDOUT,
    ) 

##### Take a look in data/genome_index 

- Each of the 5 Genome assemblies has 6 files associated
- These are Bowtie index files and are what allow for fast searching of sequences
- Without this the process of alignment would take far too long

### 2. Use Subprocess to Run Bowtie2 Alignment from CARD Genes to each of the 5 Assemblies

Documentation for Bowtie2 alignment methods (tip: check out the command line section for hints): https://bowtie-bio.sourceforge.net/bowtie2/manual.shtml#the-bowtie2-aligner

In [5]:
card_genes_path = '../data/card_data/nucleotide_combined_model.fasta'
card_genes_abspath = os.path.abspath(card_genes_path)

# Loop through our 5 test genomes
for fasta in all_genome_fastas[0:5]:

    # Access the bowtie index for the current genome
    seq_id = fasta.replace('.fna','')
    genome_index_path = '../data/genome_index/' + seq_id
    genome_index_abspath = os.path.abspath(genome_index_path)

    # Set up a path to save out the alignment results
    output_path = '../data/alignments/' + seq_id + '.sam'
    output_abspath = os.path.abspath(output_path)

    # Run the bowtie alignment (check the documentation for values)
    subprocess.run([
        'bowtie2', 
        '-x', genome_index_path, 
        '-f', card_genes_path,
        '-S', output_path,
        '-N', '1'
    ])

5113 reads; of these:
  5113 (100.00%) were unpaired; of these:
    4800 (93.88%) aligned 0 times
    313 (6.12%) aligned exactly 1 time
    0 (0.00%) aligned >1 times
6.12% overall alignment rate
5113 reads; of these:
  5113 (100.00%) were unpaired; of these:
    5016 (98.10%) aligned 0 times
    97 (1.90%) aligned exactly 1 time
    0 (0.00%) aligned >1 times
1.90% overall alignment rate
5113 reads; of these:
  5113 (100.00%) were unpaired; of these:
    4802 (93.92%) aligned 0 times
    311 (6.08%) aligned exactly 1 time
    0 (0.00%) aligned >1 times
6.08% overall alignment rate
5113 reads; of these:
  5113 (100.00%) were unpaired; of these:
    5008 (97.95%) aligned 0 times
    105 (2.05%) aligned exactly 1 time
    0 (0.00%) aligned >1 times
2.05% overall alignment rate
5113 reads; of these:
  5113 (100.00%) were unpaired; of these:
    5080 (99.35%) aligned 0 times
    33 (0.65%) aligned exactly 1 time
    0 (0.00%) aligned >1 times
0.65% overall alignment rate


### 3. Process the Alignment files to Extract the Genomes & Metadata

- Bowtie alignment produces a set of SAM files
- BAM/SAM (BAM = binary SAM) files are a common data format in bioinformatics and contain a lot of useful data
- Here you'll extract the sequence information for genes matches and save into a DataFrame

In [7]:
# Lets take a look at a single BAM file for reference
fasta_0 = all_genome_fastas[0]
fasta_0_sam_path = '../data/alignments/' + fasta_0.replace('.fna','') + '.sam'
sam_abspath = os.path.abspath(fasta_0_sam_path)
samfile = pysam.AlignmentFile(sam_abspath, "rb")

samfile_contents = samfile.fetch()

# This will loop through the first SAMFILE and stop once the first match is found
for read in samfile_contents:
    match = read.get_blocks()
    if len(match) > 0:
        break

# Take a look at the first match:
print('\nReference Sample Name (Assembly Genome):   ', read.reference_name)
print('\nCARD Gene name:  ', read.query_name)
print('\nMatch Quality:  ', read.cigarstring)
print('\nMatch Location:  ', match[0][0], 'to', match[0][1])
print('\nGenomic Sequence for Gene:  ', read.query_sequence)
print('\nGenomic Sequence for Assembly:  ', read.get_reference_sequence())


Reference Sample Name (Assembly Genome):    CYCS01000005

CARD Gene name:   gb|U00096.3|-|3324062-3324911|ARO:3003386|Ecol_folP_SLF

Match Quality:   849M

Match Location:   188190 to 189039

Genomic Sequence for Gene:   TTACTCATAGCGTTTGTTTTCCTTTGCAGACAGAGTGGCTTCCACCACCCGCATCGCTTCTACGGTTTCTTTGACGTCATGAACACGAATGATGTGCGCGCCTTGCATTGCGGCAATGACCGCACAGGCCAGACTACCGCTCAGGCGCTCGGACGGCCCCACGTTCAGCAGCTGCCCAATCATCGATTTTCGTGACATACCCACCAACAGCGGCAGGTTGAAATGGTGAAATTCAGCCAGGCGCGCCAGTAATGAATAGTTATGGGAGAGATTTTTACCGAAACCGAATCCGGGGTCGAGCAACAATTTCTCTTTTGCGATACCCGCCTGCTCGCAACGTGCTATTTGCTCAATAAAGTAGCGATTCACTTCTGCAAAGACATCGTCATACTTCGGAGCTTCCTGCATGGTTTTTGGATTTCCCTGCATATGCATCAGACAAACCGGTAAACCGGTTTCTGCAGCCGCCTCCAGAGCGCCAGGTTCGGAAAGGGAGCGGATATCATTAATAATGTGAGCGCCAACTTTCGCTGACTCACGGATGACTTCTGGTTTGGATGTATCGACTGAGATCCAGACTTCGAAGCGTTGAGCAATTGCCTCAACCACAGGAATAACACGTTGCAACTCTTCTTCAACGCTAACTTCCGCCGCCCCTGGGCGCGTGGACTCGCCACCAACGTCAATGATCGTCGCGCCAGCGTTGATCATCAGATTCGCATGTTTCACCGCATCTATCAGCGAGTTATGCGTGCCACCATCCGAAAAGGAATCAGGC

##### Summary of the Above
- The SAM file contains a record for each gene (CARD) you tried to align to the reference genome (assembly)
- Most of these are empty as the gene was not found in the assembly
- Above is the first gene that was found in the assembly
- You can see the name of the sample we're alinging against, the name of the resistance gene, the quality of the match, where in the assembly genome the gene was found (nucleotide position) and finally the sequences themselves
- If you look at the assembly sequence carefully you'll see lower case letters, these are places where the reference genome differs from the gene we were searching for but it was close enough to be considered a match

#### Now it's your turn!

In [9]:
# Loop through your 5 sample genomes
for fasta in all_genome_fastas[0:5]:
    seq_id = fasta.replace('.fna','')
    
    # Load alignment statistics using SAM
    fasta_sam_path = '../data/alignments/' + seq_id + '.sam'
    sam_abspath = os.path.abspath(fasta_sam_path)
    samfile = pysam.AlignmentFile(sam_abspath, "rb")

    # Iterate through all the potential match genes in the alignment output: if match then append to the res_genes dictionary
    res_genes = {
        'ref_name':[], 
        'contig':[], 
        'res_gene':[], 
        'match_start':[],
        'match_end':[], 
        'match_qual':[], 
        'query_str':[], 
        'ref_gene_str':[]
    }
    for read in samfile.fetch():
        match = read.get_blocks()
        if len(match) > 0:
            # When a match is found append the relevant information into the res_genes dictionary
            res_genes['ref_name'].append(seq_id)
            res_genes['contig'].append(read.reference_name)
            res_genes['res_gene'].append(read.query_name)
            res_genes['match_start'].append(match[0][0])
            res_genes['match_end'].append(match[0][-1])
            res_genes['match_qual'].append(read.cigarstring)
            res_genes['query_str'].append(read.query_sequence)
            res_genes['ref_gene_str'].append(read.get_reference_sequence())

# Generate a final dataframe from the res_genes dictionary and display
output_dataframe = pd.DataFrame(res_genes)
display(output_dataframe)

,ref_name,contig,res_gene,match_start,match_end,match_qual,query_str,ref_gene_str
0,562.7627,CYEM01000001,gb|U00096.3|-|3324062-3324911|ARO:3003386|Ecol...,570801,571650,849M,TTACTCATAGCGTTTGTTTTCCTTTGCAGACAGAGTGGCTTCCACC...,TTACTCATAGCGTTTGTTTTCCTTTGCAGACAGAGTGGCTTCCACC...
1,562.7627,CYEM01000001,gb|AP009048.1|-|3172159-3174052|ARO:3003316|Ec...,424677,426570,1893M,TTAAACCTCAATCTCCGCCATGTCGCCTTTCTCTTGCAACCAGTTG...,TcAgACCTCAATCTCtGCCATaTCGCCTTTCTCTTGCAACCAGTTG...
2,562.7627,CYEM01000001,gb|U00096.3|-|3163714-3165973|ARO:3003308|Ecol...,409145,411404,2259M,TTACTCTTCGCTATCACCGCTGCTGGCACGGCGAGGAGAGTCGATC...,TTACTCTTCGCTATCACCGCTGCTGGCACGGCGAGGAGAGTCGATC...
3,562.7627,CYEM01000001,gb|CP009072.1|+|1414817-1416077|ARO:3003775|Ec...,581828,583088,1260M,TTACTCGCCTTTCACACGCTCAATATTTGCACCTAAAGCGCGTAGT...,TTACTCGCCTTTCACACGCTCAATATTTGCACCTAAAGCGCGcAGT...
4,562.7627,CYEM01000001,gb|AE005674.2|-|3159154-3161413|ARO:3003941|Sf...,409145,411404,2259M,TTACTCTTCGCTATCACCGCTGCTGGCACGGCGAGGAGAGTCGATC...,TTACTCTTCGCTATCACCGCTGCTGGCACGGCGAGGAGAGTCGATC...
5,562.7627,CYEM01000002,gb|U00096.1|+|4277468-4277933|ARO:3003381|Ecol...,84294,84759,465M,ATGGAAAAGAAATTACCCCGCATTAAAGCGCTGCTAACCCCCGGCG...,ATGGAAAAGAAATTACCCCGCATTAAAGCGCTGCTAACCCCCGGCG...
6,562.7627,CYEM01000002,gb|U00096.3|-|4277059-4277383|ARO:3003511|Ecol...,83885,84209,324M,TTACAGGCGGTGGCGATAATCGCTGGGAGTGCGATCAAACTGCCGA...,cTACAGGCGGTGGCGATAATCGCTGGGAGTGCGATCAAACTGCCGA...
7,562.7627,CYEM01000002,gb|BA000007.3|+|5101371-5102712|ARO:3004126|Ec...,37283,38503,1220M3I3M5D3M2I110M,ATGATGATTACTCTGCGCAAACTTCCTCTGGCGGTTGCCGTCGCAG...,ATGATGATTACTCTGCGCAAACTTCCTCTGGCGGTTGCCGTCGCAG...
8,562.7627,CYEM01000001,gb|LR730402.1|+|3996374-3996926|ARO:3005042|mlaD,584503,585055,552M,TTATTTCGTTGTACCCACAGGTTCAGTGGTTTCATTATTACCTGGC...,TTATTTCGTTGTACCCACAGGTTCAGTGGTTTCATTATTACCTGGC...
9,562.7627,CYEM01000001,gb|LR730402.1|+|3994770-3995580|ARO:3005041|mlaF,585849,586659,810M,TTAACTCCCTGGTAAAAGATCAGCGTGATAATCGCCGGCAGGATAG...,TTAACTCCCTGGTAAAAGATCAGCGTGATAATCGCCGGCAGGATAG...


##### The DataFrame should look like the one below:

In [6]:
display(output_dataframe)

,ref_name,contig,res_gene,match_start,match_end,match_qual,query_str,ref_gene_str
0,562.42782,562.42782.con.0004,gb|U00096.3|-|3324062-3324911|ARO:3003386|Ecol...,96839,97688,849M,ATGAAACTCTTTGCCCAGGGTACTTCACTGGACCTTAGCCATCCTC...,ATGAAACTCTTTGCCCAGGGTACTTCACTGGACCTTAGCCATCCTC...
1,562.42782,562.42782.con.0009,gb|AP009048.1|+|3760295-3762710|ARO:3003303|Ec...,61667,64082,2415M,ATGTCGAATTCTTATGACTCCTCCAGTATCAAAGTCCTGAAAGGGC...,ATGTCGAATTCTTATGACTCCTCCAGTATCAAAGTCCTGAAAGGGC...
2,562.42782,562.42782.con.0030,gb|BA000007.3|+|4990267-4994296|ARO:3003288|Ec...,21749,25778,4029M,TTACTCGTCTTCCAGTTCGATGTTGATACCCAGCGAACGAATCTCT...,TTACTCGTCTTCCAGTTCGATGTTGATACCCAGCGAACGAATCTCT...
3,562.42782,562.42782.con.0001,gb|U00096.3|-|2336792-2339420|ARO:3003294|Ecol...,153373,156001,2628M,TTATTCTTCTTCTGGCTCGTCGTCAACGTCCACTTCCGGAGCGATT...,TTATTCTTCTTCTGGCTCGTCGTCAACGTCCACTTCCGGAGCGATT...
4,562.42782,562.42782.con.0004,gb|AP009048.1|-|3172159-3174052|ARO:3003316|Ec...,242171,244064,1893M,ATGACGCAAACTTATAACGCTGATGCCATTGAGGTACTCACCGGGC...,ATGACGCAAACTTATAACGCTGATGCCATTGAGGTACTCACCGGGC...
...,...,...,...,...,...,...,...,...
211,562.42782,562.42782.con.0020,gb|NG_049086.1|+|100-1234|ARO:3006881|EC-8,36581,37715,1134M,TTACTGTAGAGCGTTGAGAATCTGCCAGGCGGCAGTGACTCTCGCT...,TTACTGTAGAGtGTTGAGAATCTGCCAGGCGGCgGTaACTCTCGCT...
212,562.42782,562.42782.con.0004,gb|FJ768952.1|+|0-1488|ARO:3000237|TolC,237971,239459,1488M,TCAGTTACGGAAAGGGTTATGACCGTTACTGGTGGTAGTGCGTGCG...,TCAGTTACGGAAAGGGTTATGACCGTTACTGGTGGTAGTGCGTGCG...
213,562.42782,562.42782.con.0001,gb|U00096.3|-|2306971-2308615|ARO:3003952|YojI,73406,74056,650M1I3M1D990M,TTATGCCGTCCGGGCAACGGCATCACGCGAAGCGGCATCGCGCTCT...,TTATGCtGTCCGGGCAACGGCATCACGCGAAGCaGCATCGCGCTCT...
214,562.42782,562.42782.con.0008,gb|LR730402.1|+|740042-740987|ARO:3003843|leuO,78047,78992,945M,ATGCCAGAGGTACAAACAGATCATCCAGAGACGGCGGAGTTAAGCA...,ATGCCAGAGGTACAAACAGATCATCCAGAGACGGCGGAGTTAAGCA...


### Congratulations! You just performed your initial data processing steps

- This DataFrame represents a row for each resistance gene that matches to each assembly (from your sample of 5)
- We also have a lot of metadata around the gene position, quality of the match and the specific sequences
- Next week we'll be using this DataFrame as our starting point (although one which has been pre-run across all 1000 genomes) - running the full process across 1000 genomes can take a few hours if you wanted to try expanding out yourself!